In [1]:
! pip install ollama-haystack
! pip install haystack-ai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import json
from typing import List
from haystack import Pipeline, Document
from haystack.components.builders import ChatPromptBuilder
from haystack.components.retrievers import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore 
from haystack.dataclasses import ChatMessage
from haystack_integrations.components.generators.ollama import OllamaChatGenerator

In [3]:
#3a 


bpmn_examples = [
    {
        "bpmn": "<bpmn:startEvent id=\"StartEvent\" /><bpmn:task id=\"T1\" name=\"Check Credit\" /><bpmn:task id=\"T2\" name=\"Approve Loan\" />",
        "text": "The process starts with checking the customer's credit score. If the score is sufficient, the task is to approve the loan."
    },
    {
        "bpmn": "<bpmn:startEvent id=\"Start\" /><bpmn:task id=\"T3\" name=\"Process Order\" /><bpmn:task id=\"T4\" name=\"Ship Product\" /><bpmn:endEvent id=\"End\" />",
        "text": "First, process the received order. Once processing is complete, ship the final product to the customer."
    }
]

In [4]:
#3a

document_store = InMemoryDocumentStore()
documents = [
    Document(content=ex["bpmn"], meta={"text": ex["text"], "id": f"doc_{i}"})
    for i, ex in enumerate(bpmn_examples)
]
document_store.write_documents(documents)

retriever = InMemoryBM25Retriever(document_store=document_store)

In [5]:
prompt_template = [
    ChatMessage.from_system(
        """You are an expert BPMN text generator. Your task is to analyze the user-provided 
        BPMN XML fragments and create a clear, natural language description of the process.
        
        CRITICAL RULE: The tasks in your description MUST be enclosed in <bpmn:task> tags.
        Example Tag Usage: <bpmn:task>Task Name</bpmn:task>
        """
    ),
    ChatMessage.from_user(
        """
        --- Example Context (Retrieved from Document Store) ---
        BPMN Example XML: {{examples.0.content}}
        Text Example: {{examples.0.meta.text}}
        ------------------------------------------------------
        
        Now, create a textual description for the following BPMN XML fragment:
        {{query_bpmn}}
        
        Remember to use the <bpmn:task> tags for every task name.
        """
    )
]
prompt_builder = ChatPromptBuilder(template=prompt_template,required_variables=["query_bpmn", "examples"] )

In [6]:


# 2c. Matching Generator: LLM to perform the generation.
# NOTE: Replace the model and URL if necessary.
chat_generator = OllamaChatGenerator(
    model="llama3.1:8b",
    url="http://localhost:11434",
    timeout=30*60,
    generation_kwargs={"temperature": 0.3}
)

In [7]:
bpmn_pipeline = Pipeline()

# Add components
bpmn_pipeline.add_component(instance=retriever, name="retriever")
bpmn_pipeline.add_component(instance=prompt_builder, name="prompt_builder")
bpmn_pipeline.add_component(instance=chat_generator, name="generator")

# Connect components
# 1. Provide the query (the new BPMN model) to the Retriever
bpmn_pipeline.connect("retriever.documents", "prompt_builder.examples") 
# 2. Pass the retrieved documents (as 'examples') to the PromptBuilder
bpmn_pipeline.connect("prompt_builder.prompt", "generator.messages")

🚅 Components
  - retriever: InMemoryBM25Retriever
  - prompt_builder: ChatPromptBuilder
  - generator: OllamaChatGenerator
🛤️ Connections
  - retriever.documents -> prompt_builder.examples (list[Document])
  - prompt_builder.prompt -> generator.messages (list[ChatMessage])

In [8]:
# The BPMN model we want a description for
new_bpmn_query = "<bpmn:task id=\"T5\" name=\"Verify Identity\" /><bpmn:task id=\"T6\" name=\"Set up Account\" />"

# The retriever needs the actual BPMN XML to find the closest example, 
# and the prompt builder needs the XML as 'query_bpmn'.
try:
    print("Executing Pipeline...")
    results = bpmn_pipeline.run(
        data={
            "retriever": {"query": new_bpmn_query, "top_k": 1},
            "prompt_builder": {"query_bpmn": new_bpmn_query}
        }
    )

    # Print the result
    print("\n--- Pipeline Output ---")
    final_message = results["generator"]["replies"][0].content
    print(final_message)
    print("-----------------------\n")

except Exception as e:
    print(f"\nPipeline execution failed (Check Ollama connection): {e}")

Executing Pipeline...

Pipeline execution failed (Check Ollama connection): The following component failed to run:
Component name: 'generator'
Component type: 'OllamaChatGenerator'
Error: model "llama3.1:8b" not found, try pulling it first (status code: 404)
